In [2]:
%load_ext autoreload
%autoreload 2
import re
import qiskit
from qiskit import IBMQ
import random
import math
from Gate import QGate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
IBMQ.save_account('48bfe32c2f27859552a61eaf94c289db1bab171193a72ed29d1115efecfddf2d83c2ef72d0d9d259d439bd10eed366ec99a61745e3a34dac5520e8f067c048a1')

IBMQ.load_account()
provider = IBMQ.get_provider()

system = provider.get_backend('ibmq_quito')

print(system.configuration())
# print(backend)

configrc.store_credentials:WARNING:2021-10-07 16:17:31,764: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2021-10-07 16:17:32,009: Credentials are already in use. The existing account in the session will be replaced.


In [34]:
def QASMDict(gates, circuitData):
    print(gates)
    qasmDict = {} #  {'qbit number' : []}
    newQasmDict = {}
#     print(gates)
    # the count array is an array that has a number j in an index i that would represent that qubit i has j objects
    count = []
    #we will treat the circuitData list as a stack and keep an index for the current gate we are on
    dataIndex = 0
    for x in range(len(gates)):
        if x == 1 or x == 0:
            continue
        # instantiate the qasm dict with the correct number of qubits
        if gates[x][0] == 'qreg':
            for y in range(int(gates[x][1][2])):
                qasmDict[str(y)] = []
                newQasmDict[str(y)] = []
#         print(list(qasmDict.keys()))
        if len(count) == 0:
            for b in list(qasmDict.keys()):
                count.append(0)
        print("this is the count array")
        print(count)
        if gates[x][0] != 'OPENQASM' and gates[x][0] != 'include' and gates[x][0] != 'qreg' and gates[x][0] != 'creg' and gates[x][0] != 'measure':
            if len(gates[x])-1 > 1:
                # adding multi qubit gates
                if count[int(gates[x][1][2])] > count[int(gates[x][2][2])]:
                    qasmDict[str(gates[x][2][2])].append([None])
                    newQasmDict[str(gates[x][2][2])].append(QGate())
                    count[int(gates[x][2][2])] += 1
                elif count[int(gates[x][1][2])] < count[int(gates[x][2][2])]:
                    qasmDict[str(gates[x][1][2])].append([None])
                    newQasmDict[str(gates[x][1][2])].append(QGate())
                    count[int(gates[x][1][2])] += 1
                qasmDict[str(gates[x][2][2])].append(gates[x])
                qasmDict[str(gates[x][1][2])].append(gates[x])
                newQasmDict[str(gates[x][2][2])].append(QGate(gates[x][0], circuitData[dataIndex][0], str(gates[x][2][2]), str(gates[x][1][2]), isControl=True))
                newQasmDict[str(gates[x][1][2])].append(QGate(gates[x][0], circuitData[dataIndex][0], str(gates[x][2][2]), str(gates[x][1][2]), isControl=True))
                count[int(gates[x][1][2])] += 1
                count[int(gates[x][2][2])] += 1

#                 print(gates[x][2])
                dataIndex += 1
            else:
                #adding single qubit gates
                qasmDict[str(gates[x][1][2])].append(gates[x])
#                 print(newQasmDict)
#                 print(circuitData[dataIndex][0])
                newGate = QGate()
                newQasmDict[str(gates[x][1][2])].append(QGate(gates[x][0], circuitData[dataIndex][0], str(gates[x][1][2])))
                count[int(gates[x][1][2])] += 1
                dataIndex += 1
#             for z in range(len(gates[x])-1):
#                 qasmDict[str(gates[x][z+1][2])].append(gates[x])
#                 count[int(gates[x][z+1][2])] += 1
#         print(count.index(min(count)))
#         print(count.index(max(count)))
        #even out the circuitDict with empty gates
        uneven = True
        while uneven:
            uneven = False
            minInd = count.index(min(count))
            maxInd = count.index(max(count))
            if count[maxInd] > count[minInd] + 1:
                qasmDict[str(minInd)].append([None])
                newQasmDict[str(minInd)].append(QGate())
                count[minInd] += 1
                uneven = True
        print(qasmDict)
#         for a in count:
    print(newQasmDict)      
    return qasmDict, newQasmDict

def openQASM(qasm_loc):
    circ = qiskit.QuantumCircuit.from_qasm_file(qasm_loc)
    qasm_f = open(qasm_loc, "r")
    gates = []
    for line in qasm_f:
        gates.append( list(filter(None, re.split(" |,", line.strip()[:-1]))) )
#     print("These are the raw gates")
#     print(gates)
    qasmDict = QASMDict(gates, circ.data)
    qasm_f.close()
    return qasmDict


In [22]:
#MAIN SWAPS
def HHtoNone(qubit, depth, circuitDict):
    index2 = depth + 1
    if circuitDict[qubit][depth + 1] == []:
        for x in range(depth+1,len(circuitDict[qubit])):
            if circuitDict[qubit][x] != []:
                index2 = x
                break
    circuitDict[qubit][index2] = []
    circuitDict[qubit][depth] = []
    return circuitDict

def HHtoNoneObj(qubit, depth, circuitDict):
    index2 = depth + 1
    if circuitDict[qubit][depth + 1].type == None:
        for x in range(depth+1,len(circuitDict[qubit])):
            if circuitDict[qubit][x].type != None:
                index2 = x
                break
    circuitDict[qubit][index2].clear()
    circuitDict[qubit][depth].clear()
    return circuitDict


def NonetoHH(qubit, depth, circuitDict):
    qString = 'q[' + qubit + ']'
    circuitDict[qubit].insert(depth, ['h', qString])
    circuitDict[qubit].insert(depth, ['h', qString])
    return circuitDict

def NonetoHHObj(qubit, depth, circuitDict):
    qString = 'q[' + qubit + ']'
    circuitDict[qubit].insert(depth, QGate('h', None, qubit))
    circuitDict[qubit].insert(depth, QGate('h', None, qubit))
    for x in circuitDict.keys():
        if x != qubit:
            circuitDict[x].insert(depth, QGate())
            circuitDict[x].insert(depth, QGate())
    return circuitDict


# Depth is cz depth
def HZHtoCX(qubit, depth, circuitDict):
    qString = 'q[' + qubit + ']'
    circuitDict[circuitDict[qubit][depth][1][2]][depth][0] = 'cx'
    circuitDict[qubit][depth-1] = [None]
    circuitDict[qubit][depth][0] = 'cx'
    circuitDict[qubit][depth+1] = [None]
#     print(circuitDict[circuitDict[qubit][depth][1][2]][depth])
    return circuitDict

def HZHtoCXObj(qubit, depth, circuitDict):
    circuitDict[circuitDict[qubit][depth].controlBit][depth].type = 'cx'
    circuitDict[qubit][depth-1].clear()
    circuitDict[circuitDict[qubit][depth].mainBit][depth].type = 'cx'
    circuitDict[qubit][depth+1].clear()
    return circuitDict

# Depth is cx depth
def CXtoHZH(qubit, depth, circuitDict):
    qString = 'q[' + qubit + ']'
    circuitDict[qubit][depth][0] = 'cz'
    circuitDict[circuitDict[qubit][depth][1][2]][depth][0] = 'cz'
    circuitDict[qubit].insert(depth+1, ['h', qString])
    circuitDict[qubit].insert(depth, ['h', qString])
    return circuitDict

def CXtoHZHObj(qubit, depth, circuitDict):
    circuitDict[circuitDict[qubit][depth].controlBit][depth].type = 'cz'
    circuitDict[circuitDict[qubit][depth].mainBit][depth].type = 'cz'
    for x in circuitDict.keys():
        if x != qubit:
            circuitDict[x].insert(depth+1, QGate())
            circuitDict[x].insert(depth, QGate())
        else:
            circuitDict[qubit].insert(depth+1, QGate('h', None, qubit))
            circuitDict[qubit].insert(depth, QGate('h', None, qubit))    
    return circuitDict
    
def CNOTCTtoMTC(qubit, depth, circuitDict):#CNOTTCtoMCT means CNOT where passed in qubit has Control C then Target T to Mirror Target Control. Mirror is the added CNOT
    #depth is the depth of the first control
    #qubit is the qubit with the first control
    #Find the qubits that the CNOT is on
    qubit2 = circuitDict[qubit][depth].mainBit
    qubit3 = circuitDict[qubit][depth+1].controlBit
    #Insert the first big CNOT
    circuitDict[qubit2].insert(depth, QGate('cx', qubit2, qubit3))
    circuitDict[qubit3].insert(depth, QGate('cx', qubit2, qubit3))
    circuitDict[qubit].insert(depth, QGate('cx', qubit2, qubit3))
    circuitDict[qubit][depth].clear()
    #Clear the inital small CNOT
    circuitDict[qubit][depth+1].clear()
    circuitDict[qubit2][depth+1].clear()
    #Add the initial small CNOT after second initial small CNOT
    circuitDict[qubit].insert(depth+3, QGate('cx', qubit2, qubit))
    circuitDict[qubit2].insert(depth+3, QGate('cx', qubit2, qubit))
    circuitDict[qubit3].insert(depth+3, QGate('cx', qubit2, qubit))
    circuitDict[qubit3][depth+3].clear()
    return circuitDict

def CNOTTCtoMCT(qubit, depth, circuitDict):#CNOTCTtoMCT means CNOT where passed in qubit has Target T then Control C to Mirror Target Control. Mirror is the added CNOT
    #depth is the depth of the first control
    #qubit is the qubit with the first control
    #Find the qubits that the CNOT is on
    qubit2 = circuitDict[qubit][depth].controlBit
    qubit3 = circuitDict[qubit][depth+1].mainBit
    #Insert the first big CNOT
    circuitDict[qubit2].insert(depth, QGate('cx', qubit3, qubit2))
    circuitDict[qubit3].insert(depth, QGate('cx', qubit3, qubit2))
    circuitDict[qubit].insert(depth, QGate('cx', qubit3, qubit2))
    circuitDict[qubit][depth].clear()
    #Clear the inital small CNOT
    circuitDict[qubit][depth+1].clear()
    circuitDict[qubit2][depth+1].clear()
    #Add the initial small CNOT after second initial small CNOT
    circuitDict[qubit].insert(depth+3, QGate('cx', qubit, qubit2))
    circuitDict[qubit2].insert(depth+3, QGate('cx', qubit, qubit2))
    circuitDict[qubit3].insert(depth+3, QGate('cx', qubit, qubit2))
    circuitDict[qubit3][depth+3].clear()
    return circuitDict

def CNOTMTCtoCT(qubit, depth, circuitDict):
    #depth is the depth of the first control
    #qubit is the qubit with the first control
    #Find the qubits that the CNOT is on
    qubit2 = circuitDict[qubit][depth].mainBit
    qubit3 = circuitDict[qubit][depth+1].mainBit
    #clear the first CNOT
    circuitDict[qubit][depth].clear()
    circuitDict[qubit2][depth].clear()
    #clear the second CNOT
    circuitDict[qubit][depth+1].clear()
    circuitDict[qubit3][depth+1].clear()
    #Add the middle CNOT to the end
    circuitDict[qubit].insert(depth+3, QGate('cx', qubit3, qubit))
    circuitDict[qubit2].insert(depth+3, QGate('cx', qubit3, qubit))
    circuitDict[qubit3].insert(depth+3, QGate('cx', qubit3, qubit))
    circuitDict[qubit2][depth+3].clear()
    return circuitDict

def CNOTMCTtoTC(qubit, depth, circuitDict):
    #depth is the depth of the first control
    #qubit is the qubit with the first control
    #Find the qubits that the CNOT is on
    qubit2 = circuitDict[qubit][depth].mainBit
    qubit3 = circuitDict[qubit2][depth+1].controlBit
    #clear the first CNOT
    circuitDict[qubit][depth].clear()
    circuitDict[qubit2][depth].clear()
    #clear the second CNOT
    circuitDict[qubit2][depth+1].clear()
    circuitDict[qubit3][depth+1].clear()
    #Add the middle CNOT to the end
    circuitDict[qubit].insert(depth+3, QGate('cx', qubit2, qubit3))
    circuitDict[qubit2].insert(depth+3, QGate('cx', qubit2, qubit3))
    circuitDict[qubit3].insert(depth+3, QGate('cx', qubit2, qubit3))
    circuitDict[qubit][depth+3].clear()
    return circuitDict

In [23]:
def dictToQasm(circuitDict):
    outText = "OPENQASM 2.0;\ninclude \"qelib1.inc\";"
#     print(len(circuitDict.keys()))
    outText += '\nqreg q[' + str(len(circuitDict.keys())) + '];\ncreg c[' + str(len(circuitDict.keys())) + '];\n'
    
    maxind = 0
    for x in range(len(circuitDict.keys())):
#         print(x)
        if len(circuitDict[str(x)]) > len(circuitDict[str(maxind)]):
            maxind = x
    ind = 0
    while len(circuitDict[str(maxind)]) > ind:
        for y in range(len(circuitDict.keys())):
            if len(circuitDict[str(y)]) > ind:
                if circuitDict[str(y)][ind] == []:
                    continue
                elif len(circuitDict[str(y)][ind]) == 2:
                    outText += circuitDict[str(y)][ind].type + ' ' + circuitDict[str(y)][ind][1] + ';\n'
                elif len(circuitDict[str(y)][ind]) == 3:
                    if circuitDict[str(y)][ind][2][2] == str(y):
                        outText += circuitDict[str(y)][ind][0] + ' ' + circuitDict[str(y)][ind][1] + ',' + circuitDict[str(y)][ind][2] + ';\n'
        ind += 1
    for z in range(len(circuitDict.keys())):
        outText += 'measure q[' + str(z) + '] -> c[' + str(z) + '];\n'
            
    return outText


def objDictToQasm(circuitDict):
    outText = "OPENQASM 2.0;\ninclude \"qelib1.inc\";"
#     print(len(circuitDict.keys()))
    outText += '\nqreg q[' + str(len(circuitDict.keys())) + '];\ncreg c[' + str(len(circuitDict.keys())) + '];\n'
    
    maxind = 0
    for x in range(len(circuitDict.keys())):
#         print(x)
        if len(circuitDict[str(x)]) > len(circuitDict[str(maxind)]):
            maxind = x
    ind = 0
    while len(circuitDict[str(maxind)]) > ind:
        for y in range(len(circuitDict.keys())):
            if len(circuitDict[str(y)]) > ind:
                if circuitDict[str(y)][ind].type == None:
                    continue
                elif circuitDict[str(y)][ind].mainBit == str(y):
                    outText +=circuitDict[str(y)][ind].qasmString()
#                 elif circuitDict[str(y)][ind].controlBit == None:
#                     outText += circuitDict[str(y)][ind][0] + ' ' + circuitDict[str(y)][ind][1] + ';\n'
#                 else:
#                     if circuitDict[str(y)][ind][2][2] == str(y):
#                         outText += circuitDict[str(y)][ind][0] + ' ' + circuitDict[str(y)][ind][1] + ',' + circuitDict[str(y)][ind][2] + ';\n'
        ind += 1
    for z in range(len(circuitDict.keys())):
        outText += 'measure q[' + str(z) + '] -> c[' + str(z) + '];\n'
            
    return outText

In [24]:
def swapList(circuitDict):

    swapDict = {'HHtoNone': [], 'HZHtoCX': [], 'CXtoHZH': [], 'NonetoHH': []}

    for qubit in circuitDict:
        gates = circuitDict[qubit]

        # go through all the gates for a qubit
        for i in range(len(gates)):
#             print(gates[i])
            # HHtoNone
            if i < len(gates) - 1:
                if (gates[i][0] == 'h') and (gates[i+1][0] == 'h'):
                    swapDict['HHtoNone'].append([qubit, i])

            # HZHtoCX
            if i > 0 and i <= len(gates) - 1:
                if (gates[i-1][0] == 'h') and (gates[i][0] == 'cz') and (gates[i+1][0] == 'h') and (gates[i-1][-1] == gates[i][-1]):
                    if gates[i][2][2] == qubit:
                        swapDict['HZHtoCX'].append([qubit, i])

            # CXtoHZH
            if (gates[i][0] == 'cx'):
                if gates[i][2][2] == qubit:
                    swapDict['CXtoHZH'].append([qubit, i])

            # None to HH
            swapDict['NonetoHH'].append([qubit, i])
        # None to HH
        swapDict['NonetoHH'].append([qubit, len(gates)-1])
        
    return swapDict

In [25]:
def updateCircuit(circuit):
    file1 = open("temp.qasm","w+")
    file1.write(objDictToQasm(circuit))
    file1.close()
    stringCircuit, objCircuit = openQASM('temp.qasm')
    swapDict = swapList(stringCircuit)
    return (objCircuit, swapDict)
    

In [26]:
def slicer(circuitQasm, depth):
    circuit = openQASM(circuitQasm)
    base = dict(circuit)
    for qubit in base:
        base[qubit] = []
    print(circuit)
    print(base)
    leastLen = 9999999999
    maxLen = 0
    for x in circuit:
        if len(circuit[x]) < leastLen:
            leastLen = len(circuit[x])
        if len(circuit[x]) > maxLen:
            maxLen = len(circuit[x])
    if depth > leastLen or (depth == leastLen and depth == maxLen):
        return dictToQasm(circuitQasm)
    print(leastLen)
    numSubCircuits = math.floor(leastLen/depth)
    subCircuitList = []
    for subCirc in range(numSubCircuits):
        subCircuitList.append(dict(base))
        for q in subCircuitList[subCirc]:
            subCircuitList[subCirc][q] = circuit[q][0:depth]
#             print(subCircuitList)
            del circuit[q][0:depth]
    if circuit != dict(base):
        subCircuitList.append(circuit)
    qasmList = []
    print(subCircuitList)
    for dictCircuit in subCircuitList:
        qasmList.append(dictToQasm(dictCircuit))
    return qasmList
# sliced = slicer('circuit.qasm', 2)
# # print(sliced)
# circ_out = qiskit.QuantumCircuit.from_qasm_str(sliced[3])
# circ_out.draw()

    

In [35]:
stringCircuit, objCircuit = openQASM('CNOT.qasm')
print(stringCircuit)
print(objCircuit)
swapDict = swapList(stringCircuit)
print(swapDict)
# qubit, depth = random.choice(swapDict['HZHtoCX'])

# print(HZHtoCX('1', 1, circuit))

# print(CXtoHZH(qubit, depth, circuit))
# swappedCirc = HZHtoCXObj('1', 1, objCircuit)
# swappedCirc = HHtoNoneObj('0', 2, objCircuit)
# print(swappedCirc)
# updateCircuit(swappedCirc)

[['OPENQASM', '2.0'], ['include', '"qelib1.inc"'], ['qreg', 'q[3]'], ['creg', 'c[3]'], ['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]'], ['measure', 'q[0]', '->', 'c[0]'], ['measure', 'q[1]', '->', 'c[1]']]
this is the count array
[0, 0, 0]
{'0': [], '1': [], '2': []}
this is the count array
[0, 0, 0]
{'0': [], '1': [], '2': []}
this is the count array
[0, 0, 0]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]']], '2': []}
this is the count array
[1, 1, 0]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]']], '2': [[None], ['cx', 'q[1]', 'q[2]']]}
this is the count array
[1, 2, 2]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]']], '2': [[None], ['cx', 'q[1]', 'q[2]']]}
this is the count array
[1, 2, 2]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]']], '2': [[None], ['cx', 'q[1]', 'q[2]']]}
{'0': [['cx', '1', '0']], '1': [['cx', '1', '0'], ['cx', '2', '1']], '2': [[None, Non

In [37]:
circ = qiskit.QuantumCircuit.from_qasm_file('circuit.qasm')
circ.draw()

┌─┐   
q_0: ──────■──────┤M├───
     ┌───┐ │ ┌───┐└╥┘┌─┐
q_1: ┤ H ├─■─┤ H ├─╫─┤M├
     └───┘   └───┘ ║ └╥┘
c: 2/══════════════╩══╩═
                   0  1

In [13]:
circ.data

[(<qiskit.circuit.library.standard_gates.h.HGate object at 0x10cb2fca0>, [Qubit(QuantumRegister(2, 'q'), 1)], []), (<qiskit.circuit.library.standard_gates.z.CZGate object at 0x1321fdb50>, [Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)], []), (<qiskit.circuit.library.standard_gates.h.HGate object at 0x1321fd220>, [Qubit(QuantumRegister(2, 'q'), 1)], []), (<qiskit.circuit.measure.Measure object at 0x1321fd130>, [Qubit(QuantumRegister(2, 'q'), 0)], [Clbit(ClassicalRegister(2, 'c'), 0)]), (<qiskit.circuit.measure.Measure object at 0x1321fda30>, [Qubit(QuantumRegister(2, 'q'), 1)], [Clbit(ClassicalRegister(2, 'c'), 1)])]

In [14]:
print(objDictToQasm(swappedCirc))
circ_out = qiskit.QuantumCircuit.from_qasm_str(objDictToQasm(swappedCirc))
circ_out.draw()

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
cx q[0], q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];



┌─┐   
q_0: ──■──┤M├───
     ┌─┴─┐└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └───┘ ║ └╥┘
c: 2/══════╩══╩═
           0  1

In [15]:
circ = qiskit.QuantumCircuit.from_qasm_file('CNOT.qasm')
circ.draw()

┌─┐   
q_0: ──■───────┤M├───
     ┌─┴─┐     └╥┘┌─┐
q_1: ┤ X ├──■───╫─┤M├
     └───┘┌─┴─┐ ║ └╥┘
q_2: ─────┤ X ├─╫──╫─
          └───┘ ║  ║ 
c: 3/═══════════╩══╩═
                0  1

In [36]:
stringCircuit, objCircuit = openQASM('CNOT.qasm')
print(stringCircuit)
print(objCircuit)
swappedCirc = CNOTCTtoMTC('2', 0, objCircuit)
print(swappedCirc)
updateCircuit(swappedCirc)

[['OPENQASM', '2.0'], ['include', '"qelib1.inc"'], ['qreg', 'q[3]'], ['creg', 'c[3]'], ['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]'], ['measure', 'q[0]', '->', 'c[0]'], ['measure', 'q[1]', '->', 'c[1]']]
this is the count array
[0, 0, 0]
{'0': [], '1': [], '2': []}
this is the count array
[0, 0, 0]
{'0': [], '1': [], '2': []}
this is the count array
[0, 0, 0]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]']], '2': []}
this is the count array
[1, 1, 0]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]']], '2': [[None], ['cx', 'q[1]', 'q[2]']]}
this is the count array
[1, 2, 2]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]']], '2': [[None], ['cx', 'q[1]', 'q[2]']]}
this is the count array
[1, 2, 2]
{'0': [['cx', 'q[0]', 'q[1]']], '1': [['cx', 'q[0]', 'q[1]'], ['cx', 'q[1]', 'q[2]']], '2': [[None], ['cx', 'q[1]', 'q[2]']]}
{'0': [['cx', '1', '0']], '1': [['cx', '1', '0'], ['cx', '2', '1']], '2': [[None, Non

KeyError: None